# HW2

#### Machine Learning in Korea University
#### COSE362, Fall 2018
#### Due : 11/26 (TUE) 11:59 PM

#### In this assignment, you will learn various classification methods with given datasets.
* Implementation detail: Anaconda 5.3 with python 3.7
* Use given dataset. Please do not change train / valid / test split.
* Use numpy, scikit-learn, and matplotlib library
* You don't have to use all imported packages below. (some are optional). <br>
Also, you can import additional packages in "(Option) Other Classifiers" part. 
* <b>*DO NOT MODIFY OTHER PARTS OF CODES EXCEPT "Your Code Here"*</b>

In [13]:
# Basic packages
%matplotlib inline
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

# Machine Learning Models
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Additional packages
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

In [14]:
# Import your own packages if you need(only in scikit-learn, numpy, pandas).
# Your Code Here
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn import svm,grid_search
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier


#warning
## Solve
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"
#End Your Code

## Process

> 1. Load "train.csv". It includes all samples' features and labels.
> 2. Training four types of classifiers(logistic regression, decision tree, random forest, support vector machine) and <b>validate</b> it in your own way. <b>(You can't get full credit if you don't conduct validation)</b>
> 3. Optionally, if you would train your own classifier(e.g. ensembling or gradient boosting), you can evaluate your own model on the development data. <br>
> 4. <b>You should submit your predicted results on test data with the selected classifier in your own manner.</b>

## Task & dataset description
1. 6 Features (1~6)<br>
Feature 2, 4, 6 : Real-valued<br>
Feature 1, 3, 5 : Categorical <br>

2. Samples <br>
>In development set : 2,000 samples <br>
>In test set : 1,500 samples

## Load development dataset
Load your development dataset. You should read <b>"train.csv"</b>. This is a classification task, and you need to preprocess your data for training your model. <br>
> You need to use <b>1-of-K coding scheme</b>, to convert categorical features to one-hot vector. <br>
> For example, if there are 3 categorical values, you can convert these features as [1,0,0], [0,1,0], [0,0,1] by 1-of-K coding scheme. <br>

In [15]:
# For training your model, you need to convert categorical features to one-hot encoding vectors.
# Your Code Here
#load data
df_train = pd.read_csv('./data/train.csv')

#one-hot
df_train_onehot = pd.get_dummies(df_train.drop(columns=['feature2','feature4','feature6','target']))
df_train = pd.concat([df_train,df_train_onehot], axis=1)
df_train.drop(['feature1','feature3','feature5'], axis = 1, inplace = True)

#feature && label
data = df_train.drop(['target'],axis = 1,inplace = False)
target = df_train.target

#functions for cv
def calc_train_error(X_train, y_train, model):
    predictions = model.predict(X_train)
    f1_train = f1_score(y_train, predictions, average='macro')
    return f1_train

def calc_test_error(X_val, y_val, model):
    predictions = model.predict(X_val)
    f1_val = f1_score(y_val, predictions, average='macro')
    return f1_val
    
def calc_metrics(X_train, y_train, X_val, y_val, model):
    model.fit(X_train, y_train)
    train_error = calc_train_error(X_train, y_train, model)
    val_error= calc_test_error(X_val, y_val, model)
    return train_error, val_error

# End Your Code

In [16]:
df_train.columns

Index(['feature2', 'feature4', 'feature6', 'target', 'feature1_a',
       'feature1_b', 'feature1_c', 'feature1_d', 'feature3_a', 'feature3_b',
       'feature3_c', 'feature3_d', 'feature3_e', 'feature3_f', 'feature3_g',
       'feature3_h', 'feature5_a', 'feature5_b', 'feature5_c', 'feature5_d',
       'feature5_e', 'feature5_f', 'feature5_g', 'feature5_h'],
      dtype='object')

In [12]:
df_train.head()

,feature2,feature4,feature6,target,feature1_a,feature1_b,feature1_c,feature1_d,feature3_a,feature3_b,...,feature3_g,feature3_h,feature5_a,feature5_b,feature5_c,feature5_d,feature5_e,feature5_f,feature5_g,feature5_h
0,1,5,2,3,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,2,8,2,15,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2,1,3,7,6,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,5,7,13,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,3,2,4,15,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0


In [6]:
data.columns

Index(['feature2', 'feature4', 'feature6', 'feature1_a', 'feature1_b',
       'feature1_c', 'feature1_d', 'feature3_a', 'feature3_b', 'feature3_c',
       'feature3_d', 'feature3_e', 'feature3_f', 'feature3_g', 'feature3_h',
       'feature5_a', 'feature5_b', 'feature5_c', 'feature5_d', 'feature5_e',
       'feature5_f', 'feature5_g', 'feature5_h'],
      dtype='object')

In [19]:
'''
from sklearn.ensemble import ExtraTreesClassifier
# fit an Extra Trees model to the data
model = ExtraTreesClassifier()
model.fit(data,target)
# display the relative importance of each attribute
print(model.feature_importances_)
'''

[0.10876088 0.23943693 0.22982803 0.01282088 0.01304463 0.01651501
 0.02017216 0.02697308 0.02812452 0.02528525 0.02490833 0.02887034
 0.02848443 0.02743005 0.0268353  0.01995618 0.01402656 0.01520553
 0.01516925 0.01576285 0.02021265 0.02424901 0.01792813]


In [742]:
'''
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(data, target)
features = fit.transform(data)
'''

In [743]:
'''
np.set_printoptions(precision=3)
print(fit.scores_)
'''

[261.042  32.881 463.79  310.517  82.854  69.295 152.994  47.463  30.991
  48.901  15.011  27.886  25.868  51.347  12.539 173.214  52.463  50.316
  72.091  71.496  22.83   27.619 107.541]


### Logistic Regression
Train and validate your <b>logistic regression classifier</b>, and print out your validation(or cross-validation) error.
> If you want, you can use cross validation, regularization, or feature selection methods. <br>
> <b> You should use F1 score('macro' option) as evaluation metric. </b>

In [17]:
# Training your logistic regression classifier, and print out your validation(or cross-validation) error.
# Save your own model
# Your Code Here

#hyperparameter
clf = LogisticRegression()
param_grid = {'C': [0.1, 0.3, 0.5, 0.7, 1, 1.3, 1.5], 'penalty': ['l1', 'l2']}
gridsearch = GridSearchCV(clf, 
                          param_grid,
                          scoring = "f1_macro")
gridsearch.fit(data, target)

#Create Pipeline
standardizer = StandardScaler()

#Create K-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

f1_trains = []
f1_vals = []
f1_val_result = []

for train_index, val_index in kf.split(data, target):
        
    #split data
    X_train, X_val = data.iloc[train_index], data.iloc[val_index]
    y_train, y_val = target.iloc[train_index], target.iloc[val_index]
        
    #instantaite model
    #Create a pipleline that standardized, then runs logistic regression
    pipeline = make_pipeline(standardizer,
                             LogisticRegression(random_state = 42,
                                                C = gridsearch.best_params_['C'], 
                                                penalty=gridsearch.best_params_['penalty']))
        
    #calculate errors
    f1_train, f1_val = calc_metrics(X_train, y_train, X_val, y_val, pipeline)
        
    #append tp appropriate list
    f1_trains.append(f1_train)
    f1_vals.append(f1_val)
    
    f1_val_result.append(np.mean(f1_vals))
    
#generate report
    print('mean(f1_train): {:7} | mean(f1_validation): {}'.
            format(
            round(np.mean(f1_trains),4),
            round(np.mean(f1_vals),4)
            ))
    
#print(np.mean(f1_vals))
print("======================")
print("validation f1_score")
print(np.mean(f1_val_result))
print("======================")
# End Your Code

mean(f1_train):  0.3571 | mean(f1_validation): 0.242
mean(f1_train):  0.3663 | mean(f1_validation): 0.2446
mean(f1_train):  0.3698 | mean(f1_validation): 0.2452
mean(f1_train):  0.3656 | mean(f1_validation): 0.2471
mean(f1_train):  0.3664 | mean(f1_validation): 0.2473
validation f1_score
0.24523288539881988


### Decision Tree
Train and validate your <b>decision tree classifier</b>, and print out your validation(or cross-validation) error.
> If you want, you can use cross validation, regularization, or feature selection methods. <br>
> <b> You should use F1 score('macro' option) as evaluation metric. </b>

In [18]:
# Training your decision tree classifier, and print out your validation(or cross-validation) error.
# Save your own model
# Your Code Here

#Create Pipeline
standardizer = StandardScaler()

#Create K-fold cross-validation

kf = KFold(n_splits=5, shuffle=True, random_state=42)

f1_trains = []
f1_vals = []
f1_val_result = []

for train_index, val_index in kf.split(data, target):       
    #split data
    X_train, X_val = data.iloc[train_index], data.iloc[val_index]
    y_train, y_val = target.iloc[train_index], target.iloc[val_index]
        
    #instantaite model
    #Create a pipleline that standardized, then runs logistic regression
    pipeline = make_pipeline(standardizer,
                            DecisionTreeClassifier(random_state = 42))
        
    #calculate errors
    f1_train, f1_val = calc_metrics(X_train, y_train, X_val, y_val, pipeline)
        
    #append tp appropriate list
    f1_trains.append(f1_train)
    f1_vals.append(f1_val)
    
    f1_val_result.append(np.mean(f1_vals))
    
    #generate report
    print('mean(f1_train): {:7} | mean(f1_validation): {} '.
        format(
                round(np.mean(f1_trains),4),
                round(np.mean(f1_vals),4)
                ))
    
print("======================")
print("validation f1_score")
print(np.mean(f1_val_result))
print("======================")

# End Your Code

mean(f1_train):     1.0 | mean(f1_validation): 0.3403 
mean(f1_train):     1.0 | mean(f1_validation): 0.3566 
mean(f1_train):     1.0 | mean(f1_validation): 0.3644 
mean(f1_train):     1.0 | mean(f1_validation): 0.3776 
mean(f1_train):     1.0 | mean(f1_validation): 0.3751 
validation f1_score
0.3628007526976968


### Random Forest
Train and validate your <b>random forest classifier</b>, and print out your validation(or cross-validation) error.
> If you want, you can use cross validation, regularization, or feature selection methods. <br>
> <b> You should use F1 score('macro' option) as evaluation metric. </b>

In [19]:
# Training your random forest classifier, and print out your validation(or cross-validation) error.
# Save your own model
# Your Code Here

#Create Pipeline
standardizer = StandardScaler()

#Create K-fold cross-validation

kf = KFold(n_splits=5, shuffle=True, random_state=42)

f1_trains = []
f1_vals = []
f1_val_result = []

for train_index, val_index in kf.split(data, target):       
    #split data
    X_train, X_val = data.iloc[train_index], data.iloc[val_index]
    y_train, y_val = target.iloc[train_index], target.iloc[val_index]
        
    #instantaite model
    #Create a pipleline that standardized, then runs logistic regression
    pipeline = make_pipeline(standardizer,
                            RandomForestClassifier(n_estimators=400,
                                                   random_state=42))
        
    #calculate errors
    f1_train, f1_val = calc_metrics(X_train, y_train, X_val, y_val, pipeline)
        
    #append tp appropriate list
    f1_trains.append(f1_train)
    f1_vals.append(f1_val)
    
    f1_val_result.append(np.mean(f1_vals))
    
    #generate report
    print('mean(f1_train): {:7} | mean(f1_validation): {} '.
        format(
                round(np.mean(f1_trains),4),
                round(np.mean(f1_vals),4)
                ))
    
print("======================")
print("validation f1_score")
print(np.mean(f1_val_result))
print("======================")

# End Your Code

mean(f1_train):     1.0 | mean(f1_validation): 0.5132 
mean(f1_train):     1.0 | mean(f1_validation): 0.4745 
mean(f1_train):     1.0 | mean(f1_validation): 0.4431 
mean(f1_train):     1.0 | mean(f1_validation): 0.4477 
mean(f1_train):     1.0 | mean(f1_validation): 0.4435 
validation f1_score
0.464401899460837


### Support Vector Machine
Train and validate your <b>support vector machine classifier</b>, and print out your validation(or cross-validation) error.
> If you want, you can use cross validation, regularization, or feature selection methods. <br>
> <b> You should use F1 score('macro' option) as evaluation metric. </b>

In [21]:
svc_param_selection(data, target, 5)

{'C': 10, 'gamma': 0.1}

In [20]:
# Training your support vector machine classifier, and print out your validation(or cross-validation) error.
# Save your own model
# Your Code Here

#hyperparameter
def svc_param_selection(X, y, nfolds):
    
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(
                               svm.SVC(kernel='rbf',random_state = 42),
                               param_grid, 
                               cv = nfolds,
                               scoring='f1_macro')
    grid_search.fit(X, y)
    grid_search.best_params_
    
    return grid_search.best_params_


#Create Pipeline
standardizer = StandardScaler()

#Create K-fold cross-validation

kf = KFold(n_splits=5, shuffle=True, random_state=42)

f1_trains = []
f1_vals = []
f1_val_result = []

for train_index, val_index in kf.split(data, target):       
    #split data
    X_train, X_val = data.iloc[train_index], data.iloc[val_index]
    y_train, y_val = target.iloc[train_index], target.iloc[val_index]
        
    #instantaite model
    #Create a pipleline that standardized, then runs logistic regression
    pipeline = make_pipeline(standardizer,
                            SVC(C = 10, 
                                gamma = 0.1, 
                                kernel='rbf',
                                random_state = 42))
        
    #calculate errors
    f1_train, f1_val = calc_metrics(X_train, y_train, X_val, y_val, pipeline)
        
    #append tp appropriate list
    f1_trains.append(f1_train)
    f1_vals.append(f1_val)
    
    f1_val_result.append(np.mean(f1_vals))
    
    #generate report
    print('mean(f1_train): {:7} | mean(f1_validation): {} '.
        format(
                round(np.mean(f1_trains),4),
                round(np.mean(f1_vals),4)
                ))
    
print("======================")
print("validation f1_score")
print(np.mean(f1_val_result))
print("======================")
# End Your Code

mean(f1_train):  0.9009 | mean(f1_validation): 0.3574 
mean(f1_train):  0.8915 | mean(f1_validation): 0.3595 
mean(f1_train):  0.9002 | mean(f1_validation): 0.3459 
mean(f1_train):  0.8956 | mean(f1_validation): 0.339 
mean(f1_train):  0.8913 | mean(f1_validation): 0.3446 
validation f1_score
0.3492606240610674


### (Option) Other Classifiers.
Train and validate other classifiers by your own manner.
> <b> If you need, you can import other models only in this cell, only in scikit-learn. </b>

In [59]:

parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }

clf = GridSearchCV(GradientBoostingClassifier(), parameters, cv = 5, n_jobs=-1)
clf.fit(data, target)
print(clf.score(trainX, trainY))
print(clf.best_params_)

KeyboardInterrupt: 

In [26]:
# If you need additional packages, import your own packages below.
# Your Code Here


#Create Pipeline
standardizer = StandardScaler()

#Create K-fold cross-validation

kf = KFold(n_splits=5, shuffle=True, random_state=42)

f1_trains = []
f1_vals = []
f1_val_result = []

for train_index, val_index in kf.split(data, target):       
    #split data
    X_train, X_val = data.iloc[train_index], data.iloc[val_index]
    y_train, y_val = target.iloc[train_index], target.iloc[val_index]
        
    #instantaite model
    #Create a pipleline that standardized, then runs logistic regression
    pipeline = make_pipeline(standardizer,
                             GradientBoostingClassifier(
                                                       random_state = 42,
                                                       min_samples_split = 10,
                                                       n_estimators=400,
                                                       max_depth = 5,
                                                       subsample = 0.8,
                                                       learning_rate = 0.1,
                                                       max_features = 'sqrt'
                                                      )
                            )
        
    #calculate errors
    f1_train, f1_val = calc_metrics(X_train, y_train, X_val, y_val, pipeline)
        
    #append tp appropriate list
    f1_trains.append(f1_train)
    f1_vals.append(f1_val)
    
    f1_val_result.append(np.mean(f1_vals))
    
    #generate report
    print('mean(f1_train): {:7} | mean(f1_validation): {} '.
        format(
                round(np.mean(f1_trains),4),
                round(np.mean(f1_vals),4)
                ))
    
print("======================")
print("validation f1_score")
print(np.mean(f1_val_result))
print("======================")

# End Your Code

mean(f1_train):     1.0 | mean(f1_validation): 0.5228 
mean(f1_train):     1.0 | mean(f1_validation): 0.5176 
mean(f1_train):     1.0 | mean(f1_validation): 0.4874 
mean(f1_train):     1.0 | mean(f1_validation): 0.4689 
mean(f1_train):     1.0 | mean(f1_validation): 0.4734 
validation f1_score
0.4940271842008638


좀 걸리는데 성능은 확 올라갈거야 기다려봐요 :)

In [29]:
# If you need additional packages, import your own packages below.
# Your Code Here

#Create Pipeline
standardizer = StandardScaler()

#Create K-fold cross-validation

kf = KFold(n_splits=5, shuffle=True, random_state=42)

f1_trains = []
f1_vals = []
f1_val_result = []

for train_index, val_index in kf.split(data, target):       
    #split data
    X_train, X_val = data.iloc[train_index], data.iloc[val_index]
    y_train, y_val = target.iloc[train_index], target.iloc[val_index]
        
    #instantaite model
    #Create a pipleline that standardized, then run
    pipeline = make_pipeline(standardizer,
                             GradientBoostingClassifier(
                                                       random_state = 42,
                                                       n_estimators = 400,  
                                                       subsample = 0.8
                                                      )
                            )
        
    #calculate errors
    f1_train, f1_val = calc_metrics(X_train, y_train, X_val, y_val, pipeline)
        
    #append tp appropriate list
    f1_trains.append(f1_train)
    f1_vals.append(f1_val)
    
    f1_val_result.append(np.mean(f1_vals))
    
    #generate report
    print('mean(f1_train): {:7} | mean(f1_validation): {} '.
        format(
                round(np.mean(f1_trains),4),
                round(np.mean(f1_vals),4)
                ))
    
print("======================")
print("validation f1_score")
print(np.mean(f1_val_result))
print("======================")

# End Your Code

mean(f1_train):  0.9947 | mean(f1_validation): 0.5837 
mean(f1_train):  0.9943 | mean(f1_validation): 0.5369 
mean(f1_train):   0.994 | mean(f1_validation): 0.4975 
mean(f1_train):  0.9944 | mean(f1_validation): 0.4747 
mean(f1_train):  0.9946 | mean(f1_validation): 0.4769 
validation f1_score
0.5139382956863877


In [ ]:
pipeline = make_pipeline(standardizer,
                         GradientBoostingClassifier(
                                                    random_state = 42,
                                                    n_estimators = 400,  
                                                    subsample = 0.8
                                                    )
                        )

## Submit your prediction on the test data.

* Select your model and explain it briefly.
* You should read <b>"test.csv"</b>.
* Prerdict your model in array form.
* Prediction example <br>
[2, 6, 14, 8, $\cdots$]
* We will rank your result by <b>F1 metric(with 'macro' option)</b>.
* <b> If you don't submit prediction file or submit it in wrong format, you can't get the point for this part.

In [122]:
# Explain your final model





In [133]:
# Load test dataset.
# Your Code Here
df_test = pd.read_csv('./data/test.csv')
# End Your Code

In [134]:
# Predict target class
# Make variable "my_answer", type of array, and fill this array with your class predictions.
# Modify file name into your student number and your name.
# Your Code Here
my_answer = []

#one-hot
df_test_onehot = pd.get_dummies(df_test.drop(columns=['feature2','feature4','featrure6']))
df_test = pd.concat([df_test,df_test_onehot], axis=1)
df_test.drop(['feature1','feature3','feature5'], axis = 1, inplace = True)

logreg_test = logreg.predict(df_test)

file_name = "HW2_2016320120_정소영.csv"
# End Your Code

In [135]:
logreg_test

array([6, 6, 6, ..., 1, 6, 2], dtype=int64)

In [136]:
# This section is for saving predicted answers. DO NOT MODIFY.
pd.Series(my_answer).to_csv("./data/" + file_name, header=None, index=None)